This block read data from a demo session. Mouse #27049, 20220516, session 1

In [1]:
import numpy as np
import os
import pandas as pd
import pickle
import time

from mazepy.datastruc.variables import KilosortSpikeTrain, VariableBin
from replay.preprocess.read import read_npxdata 
from replay.preprocess.neuact import get_within_trial_spike_index
from replay.preprocess.behav import transform_time_format
from replay.local_path import f1_behav as f

i = 0

with open(f['Trace Behav File'][i], 'rb') as handle:
    trace = pickle.load(handle)

trace.update(read_npxdata(dir_name=f['npx_folder'][i]))

In [4]:

print(type(f['npx_init_time'][i]), type(f['behav_init_time'][i]))
npx_init_time = transform_time_format(f['npx_init_time'][i])[0]
behav_init_time = transform_time_format(f['behav_init_time'][i])[0]
behav_time = trace['video_time'] - npx_init_time + behav_init_time
    
within_trial_spike_index = get_within_trial_spike_index(
    trace['spike_times'],
    behav_time[trace['onset_frames']],
    behav_time[trace['end_frames']]
)


spikes_remain = trace['spike_clusters'][within_trial_spike_index]
spike_time_remain = trace['spike_times'][within_trial_spike_index]
frequency = trace['dominant_freq_filtered'][trace['dominant_freq_filtered'] > 0]

from replay.preprocess.neuact import get_spike_related_freq

spike_freq = get_spike_related_freq(
    spike_time_remain, 
    behav_time,
    trace['onset_frames'],
    trace['end_frames'],
    trace['dominant_freq_filtered']
).astype(np.int64) - 23 # set to 0

assert spike_time_remain.shape[0] == spike_freq.shape[0]

neural_activity = KilosortSpikeTrain(
    activity=spikes_remain,
    time_stamp=spike_time_remain * 1000,
    variable=VariableBin(spike_freq)
)

<class 'str'> <class 'str'>


In [5]:
neural_activity.activity
neural_activity.calc_firing_rate(
    nbins = (np.max(spike_freq), )
)
print(neural_activity.firing_rate)

[[0.02762875 0.22766554 0.30836235 ... 0.         0.         0.        ]
 [0.05525751 0.39841469 0.22025882 ... 0.         0.         0.        ]
 [0.49731756 1.70749152 2.46689877 ... 0.         0.71544405 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.00920958 0.         0.04405176 ... 0.         0.         0.        ]]
